In [1]:
!pip install finnhub-python pandas openpyxl xlsxwriter

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import finnhub
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import datetime
import io
import os
import time

# Finnhub API setup
FINNHUB_API_KEY = "API-key"  # Replace with your actual key
finnhub_client = finnhub.Client(api_key=FINNHUB_API_KEY)

# ✅ Quick test to confirm connection
try:
    profile = finnhub_client.company_profile2(symbol="AAPL")
    print("✅ Finnhub connection working:", profile["name"])
except Exception as e:
    print(f"❌ Error connecting to Finnhub: {e}")

if tf.config.list_physical_devices('GPU'):
    print("🚀 Using GPU for training!")
else:
    print("⚠️ No GPU found. Running on CPU.")


✅ Finnhub connection working: Apple Inc
⚠️ No GPU found. Running on CPU.


In [3]:
def fetch_stock_data(symbol, interval="daily", lookback_days=365):
    resolution_map = {"daily": "D", "weekly": "W"}
    resolution = resolution_map.get(interval, "D")
    end_time = int(time.time())
    start_time = int((datetime.datetime.now() - datetime.timedelta(days=lookback_days)).timestamp())
    try:
        res = finnhub_client.stock_candles(symbol, resolution, start_time, end_time)
        if res['s'] != 'ok':
            print(f"⚠️ No data returned for {symbol}")
            return pd.DataFrame()
        df = pd.DataFrame({
            'Date': pd.to_datetime(res['t'], unit='s'),
            'Open': res['o'],
            'High': res['h'],
            'Low': res['l'],
            'Close': res['c'],
            'Volume': res['v'],
        }).set_index('Date').sort_index()
        print(f"✅ Finnhub data loaded for {symbol}")
        
        # ✅ Save preview of fetched data
        preview_path = f"checkpoints/fetch_{symbol}.csv"
        os.makedirs("checkpoints", exist_ok=True)
        df.head().to_csv(preview_path)
        print(f"📁 Preview saved to {preview_path}")

        return df

    except Exception as e:
        print(f"⚠️ Error fetching data for {symbol}: {e}")
        return pd.DataFrame()

In [4]:
def load_stock_list(filename="US_MK_Screener.xlsx"):
    df_tickers = pd.read_excel(filename)
    stock_list = df_tickers["Ticker"].dropna().tolist()
    print("✅ Stock List Loaded:", stock_list)
    return stock_list

SEQUENCE_LENGTH = 30

def prepare_data(df, sequence_length=SEQUENCE_LENGTH):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df[['Close']])
    X, y = [], []
    for i in range(sequence_length, len(scaled_data)):
        X.append(scaled_data[i-sequence_length:i])
        y.append(scaled_data[i, 0])
    return np.array(X).reshape(-1, sequence_length, 1), np.array(y), scaler

def build_optimized_cnn_lstm(input_shape=(30, 1)):
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        LSTM(50, return_sequences=False),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

def predict_future_prices(model, last_sequence, scaler, ticker, future_steps=30, frequency="daily"):
    future_predictions = []
    input_seq = last_sequence.copy()
    
    for _ in range(future_steps):
        pred_price = model.predict(input_seq.reshape(1, *input_seq.shape))[0, 0]
        future_predictions.append(pred_price)
        input_seq = np.roll(input_seq, -1, axis=0)
        input_seq[-1, 0] = pred_price

    future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1)).flatten()

    last_date = datetime.datetime.today()
    future_dates = [last_date + datetime.timedelta(days=i+1) for i in range(future_steps)] \
        if frequency == "daily" else \
        [last_date + datetime.timedelta(weeks=i+1) for i in range(future_steps)]

    buy_sell_signals = ["Hold"] * future_steps
    for i in range(1, future_steps):
        if future_predictions[i] > future_predictions[i-1]:
            buy_sell_signals[i] = "Buy"
        elif future_predictions[i] < future_predictions[i-1]:
            buy_sell_signals[i] = "Sell"

    # ✅ Trend logic (INSIDE the function)
    trend = "Up" if future_predictions[-1] > future_predictions[0] else "Down"
    trend_col = [trend] * future_steps

    return pd.DataFrame({
        "Ticker": ticker,
        "Date": future_dates,
        "Buy/Sell": buy_sell_signals,
        "Predicted Price": future_predictions,
        "Trend": trend_col
    })


def generate_stock_chart(df, predicted_df, ticker):
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(df.index, df["Close"], label="Historical Price", color="blue", alpha=0.7)
    ax.plot(predicted_df["Date"], predicted_df["Predicted Price"], label="Predicted Prices", color="red", linestyle="dashed")
    ax.set_title(f"{ticker} Stock Price & Predictions")
    ax.set_xlabel("Date")
    ax.set_ylabel("Stock Price")
    ax.legend()
    img_data = io.BytesIO()
    plt.savefig(img_data, format="png")
    plt.close()
    img_data.seek(0)
    return img_data

# ✅ 🔽 Place evaluate_minervini() here 🔽
def evaluate_minervini(df):
    try:
        df["SMA_50"] = df["close"].rolling(window=50).mean()
        df["SMA_150"] = df["close"].rolling(window=150).mean()
        df["SMA_200"] = df["close"].rolling(window=200).mean()

        currentClose = df["close"].iloc[-1]
        ma50 = df["SMA_50"].iloc[-1]
        ma150 = df["SMA_150"].iloc[-1]
        ma200 = df["SMA_200"].iloc[-1]
        ma200_20 = df["SMA_200"].iloc[-20] if len(df) >= 220 else ma200
        low52 = df["low"].iloc[-260:].min()
        high52 = df["high"].iloc[-260:].max()

        conditions = [
            currentClose > ma150 > ma200,
            ma150 > ma200,
            ma200 > ma200_20,
            ma50 > ma150 > ma200,
            currentClose > ma50,
            currentClose >= (1.3 * low52),
            currentClose >= (0.75 * high52)
        ]

        if all(conditions):
            return "✅ Passed Minervini"
        else:
            failed = [str(i+1) for i, c in enumerate(conditions) if not c]
            return f"❌ Failed: {', '.join(failed)}"

    except Exception:
        return "❌ Minervini check failed (data too short or missing)"

def save_results_to_excel(trades, charts, filename="AI_Evaluation_Results.xlsx"):
    with pd.ExcelWriter(filename, engine="xlsxwriter") as writer:
        # ✅ Updated columns to include Trend and Minervini
        pd.DataFrame(trades, columns=["Ticker", "Date", "Buy/Sell", "Predicted Price", "Trend", "Minervini"]) \
            .to_excel(writer, sheet_name="Predictions", index=False)

        for ticker, img_data in charts.items():
            sheet = writer.book.add_worksheet(ticker)
            sheet.insert_image(0, 0, f"{ticker}.png", {"image_data": img_data})

    print(f"📁 Predictions & Charts saved to: {filename}")

In [5]:
# ✅ Load tickers
stock_list = load_stock_list("US_MK_Screener.xlsx")
trades, charts = [], {}

# ✅ DEBUG: Limit to first 5 stocks
try:
    stock_list = stock_list[:5]
    print("✅ Debug mode: Only processing first 5 tickers:", stock_list)
except Exception as e:
    print("⚠️ Debug mode skipped:", e)

# ✅ Define fetch function (outside the loop)
def fetch_stock_data(ticker, interval="daily", from_days=500):
    resolution_map = {"daily": "D", "weekly": "W"}
    resolution = resolution_map.get(interval, "D")
    end_time = int(time.time())
    start_time = int((datetime.datetime.now() - datetime.timedelta(days=from_days)).timestamp())

    try:
        res = finnhub_client.stock_candles(ticker, resolution, start_time, end_time)
        if res['s'] != 'ok':
            print(f"⚠️ No data returned for {ticker}")
            return pd.DataFrame()

        df = pd.DataFrame({
            'Date': pd.to_datetime(res['t'], unit='s'),
            'Open': res['o'],
            'High': res['h'],
            'Low': res['l'],
            'Close': res['c'],
            'Volume': res['v'],
        }).set_index('Date').sort_index()

        print(f"✅ Finnhub data loaded for {ticker} ({len(df)} rows)")
        
        # Optional: Save preview
        os.makedirs("checkpoints", exist_ok=True)
        df.head().to_csv(f"checkpoints/{ticker}_preview.csv")

        return df

    except Exception as e:
        print(f"⚠️ Error fetching data for {ticker}: {e}")
        return pd.DataFrame()

# ✅ Main loop
for ticker in stock_list:
    print(f"\n🚀 Processing {ticker}...")

    df = fetch_stock_data(ticker)
    if df.empty:
        continue

    X, y, scaler = prepare_data(df)
    model_path = f"{ticker}_cnn_lstm_model.h5"

    if os.path.exists(model_path):
        print(f"📂 Loading pre-trained model for {ticker}")
        model = load_model(model_path)
    else:
        print(f"🛠 Training new model for {ticker}...")
        model = build_optimized_cnn_lstm(input_shape=(30, 1))
        early_stop = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
        model.fit(X, y, epochs=20, batch_size=16, verbose=0, callbacks=[early_stop])
        model.save(model_path)

    predicted_df = predict_future_prices(model, X[-1], scaler, ticker)

    # ✅ Add Minervini status
    minervini_result = evaluate_minervini(df)
    predicted_df["Minervini"] = minervini_result

    trades.extend(predicted_df.values.tolist())
    charts[ticker] = generate_stock_chart(df, predicted_df, ticker)
    print(f"✅ Completed prediction for {ticker}")

# ✅ Save everything
def save_results_to_excel(trades, charts, filename="US_MK_check_point.xlsx"):
    with pd.ExcelWriter(filename, engine="xlsxwriter") as writer:
        df = pd.DataFrame(trades, columns=["Ticker", "Date", "Buy/Sell", "Predicted Price", "Trend", "Minervini"])
        df.to_excel(writer, sheet_name="Predictions", index=False)

        for ticker, img_data in charts.items():
            sheet = writer.book.add_worksheet(ticker)
            sheet.insert_image(0, 0, f"{ticker}.png", {"image_data": img_data})

        summary_data = {
            "Minervini Passed": (df["Minervini"] == "✅ Passed Minervini").sum(),
            "Minervini Failed": (df["Minervini"] != "✅ Passed Minervini").sum(),
            "Trend Up": (df["Trend"].str.lower() == "up").sum(),
            "Trend Down": (df["Trend"].str.lower() == "down").sum(),
            "Buy Signals": (df["Buy/Sell"] == "Buy").sum(),
            "Sell Signals": (df["Buy/Sell"] == "Sell").sum(),
            "Hold Signals": (df["Buy/Sell"] == "Hold").sum()
        }
        pd.DataFrame(list(summary_data.items()), columns=["Metric", "Count"]).to_excel(writer, sheet_name="Summary", index=False)

    print(f"📁 File saved to: {filename}")

# ✅ Save now
save_results_to_excel(trades, charts)


✅ Stock List Loaded: ['TRU', 'FIS', 'PLNT', 'CTAS', 'LYV', 'AZEK', 'FAF', 'WY', 'MOS', 'DXCM', 'NFLX', 'CTVA', 'EFX', 'ALC', 'SYK', 'PWR', 'HWM', 'CMG', 'AGI', 'AMT', 'DRS', 'CHD', 'FTNT', 'ARE', 'LTH', 'ADC', 'SFM', 'KVUE', 'ORLA', 'CDE', 'JNPR', 'COR', 'FAST', 'CPRT', 'CRM', 'GT', 'FND', 'NTR', 'RVTY', 'MNST', 'IFNNY', 'CGNX', 'DV', 'FI', 'CLX', 'NTDOY', 'UNH', 'WMT', 'CHWY', 'BEKE', 'KEYS', 'BAM', 'CAG', 'BIRK', 'AMGN', 'NVDA', 'SPGI', 'RSG', 'COO', 'CWEN', 'RELX', 'HBM', 'NDAQ', 'EXLS', 'LUV', 'CBRE', 'MA', 'IBM', 'IDXX', 'KIM', 'EA', 'ENTG', 'NWSA', 'RTX', 'CTRE', 'DHR', 'BRBR', 'RGLD', 'IR', 'OLLI', 'VMC', 'DD', 'BX', 'ULS', 'CXW', 'HLT', 'MAA', 'BRO', 'PAGP', 'ELS', 'APH', 'V', 'PM', 'DINO', 'RRX', 'WM', 'BEPC', 'BDX', 'ETR', 'AMZN', 'KDP', 'SW', 'EXR', 'BGC', 'AEM', 'ANET', 'TAK', 'ICE', 'AMH', 'LIN', 'SBRA', 'ASML', 'RCRUY', 'ZWS', 'SHW', 'ECL', 'ELF', 'JBL', 'SBAC', 'TU', 'TTEK', 'AAPL', 'MSFT', 'SWK', 'ADP', 'SNPS', 'ORCL', 'USFD', 'KKR', 'WMB', 'PSN', 'BURL', 'TT', 'GXO', '

In [ ]:
# ✅ Actual Run for all stock
stock_list = load_stock_list("US_MK_Screener.xlsx")
trades, charts = [], {}

print("🚀 Running for all stocks in the list...")

for ticker in stock_list:
    print(f"\n🚀 Processing {ticker}...")
    df = fetch_stock_data(ticker, interval="daily", from_days=500)
    if df.empty:
        print(f"⚠️ Skipping {ticker} due to missing data")
        continue

    X, y, scaler = prepare_data(df)
    model_path = f"{ticker}_cnn_lstm_model.h5"

    if os.path.exists(model_path):
        print(f"📂 Loading pre-trained model for {ticker}")
        model = load_model(model_path)
    else:
        print(f"🛠 Training new model for {ticker}...")
        model = build_optimized_cnn_lstm(input_shape=(30, 1))
        early_stop = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
        model.fit(X, y, epochs=20, batch_size=16, verbose=1, callbacks=[early_stop])
        model.save(model_path)

    predicted_df = predict_future_prices(model, X[-1], scaler, ticker)
    print(predicted_df.head())  # Show first few predictions for quick debug

    # ✅ Add Minervini status
    minervini_result = evaluate_minervini(df)
    predicted_df["Minervini"] = minervini_result

    trades.extend(predicted_df.values.tolist())
    charts[ticker] = generate_stock_chart(df, predicted_df, ticker)
    print(f"✅ Completed AI trading & prediction for {ticker}!")
    

# ✅ Exporting results
def save_results_to_excel(trades, charts, filename="AI_Trading_Results.xlsx"):
    with pd.ExcelWriter(filename, engine="xlsxwriter") as writer:
        # ✅ Save main prediction table
        df = pd.DataFrame(trades, columns=["Ticker", "Date", "Buy/Sell", "Predicted Price", "Trend", "Minervini"])
        df.to_excel(writer, sheet_name="Predictions", index=False)

        # ✅ Save charts to individual sheets
        for ticker, img_data in charts.items():
            sheet = writer.book.add_worksheet(ticker)
            sheet.insert_image(0, 0, f"{ticker}.png", {"image_data": img_data})

        # ✅ Add summary sheet
        summary_data = {
            "Minervini Passed": (df["Minervini"] == "✅ Passed Minervini").sum(),
            "Minervini Failed": (df["Minervini"] != "✅ Passed Minervini").sum(),
            "Trend Up": (df["Trend"].str.lower() == "up").sum(),
            "Trend Down": (df["Trend"].str.lower() == "down").sum(),
            "Buy Signals": (df["Buy/Sell"] == "Buy").sum(),
            "Sell Signals": (df["Buy/Sell"] == "Sell").sum(),
            "Hold Signals": (df["Buy/Sell"] == "Hold").sum()
        }

        summary_df = pd.DataFrame(list(summary_data.items()), columns=["Metric", "Count"])
        summary_df.to_excel(writer, sheet_name="Summary", index=False)

    print(f"📁 Predictions, charts & summary saved to: {filename}")
    print("🎉 All stocks processed successfully!")

# ✅ Call the function AFTER defining it
save_results_to_excel(trades, charts)


✅ Stock List Loaded: ['TRU', 'FIS', 'PLNT', 'CTAS', 'LYV', 'AZEK', 'FAF', 'WY', 'MOS', 'DXCM', 'NFLX', 'CTVA', 'EFX', 'ALC', 'SYK', 'PWR', 'HWM', 'CMG', 'AGI', 'AMT', 'DRS', 'CHD', 'FTNT', 'ARE', 'LTH', 'ADC', 'SFM', 'KVUE', 'ORLA', 'CDE', 'JNPR', 'COR', 'FAST', 'CPRT', 'CRM', 'GT', 'FND', 'NTR', 'RVTY', 'MNST', 'IFNNY', 'CGNX', 'DV', 'FI', 'CLX', 'NTDOY', 'UNH', 'WMT', 'CHWY', 'BEKE', 'KEYS', 'BAM', 'CAG', 'BIRK', 'AMGN', 'NVDA', 'SPGI', 'RSG', 'COO', 'CWEN', 'RELX', 'HBM', 'NDAQ', 'EXLS', 'LUV', 'CBRE', 'MA', 'IBM', 'IDXX', 'KIM', 'EA', 'ENTG', 'NWSA', 'RTX', 'CTRE', 'DHR', 'BRBR', 'RGLD', 'IR', 'OLLI', 'VMC', 'DD', 'BX', 'ULS', 'CXW', 'HLT', 'MAA', 'BRO', 'PAGP', 'ELS', 'APH', 'V', 'PM', 'DINO', 'RRX', 'WM', 'BEPC', 'BDX', 'ETR', 'AMZN', 'KDP', 'SW', 'EXR', 'BGC', 'AEM', 'ANET', 'TAK', 'ICE', 'AMH', 'LIN', 'SBRA', 'ASML', 'RCRUY', 'ZWS', 'SHW', 'ECL', 'ELF', 'JBL', 'SBAC', 'TU', 'TTEK', 'AAPL', 'MSFT', 'SWK', 'ADP', 'SNPS', 'ORCL', 'USFD', 'KKR', 'WMB', 'PSN', 'BURL', 'TT', 'GXO', '

✅ Completed AI trading & prediction for LYV!

🚀 Processing AZEK...
✅ Finnhub data loaded for AZEK (340 rows)
📂 Loading pre-trained model for AZEK
1/1 [==============================] - 0s 33ms/step
  Ticker                       Date Buy/Sell  Predicted Price Trend
0   AZEK 2025-04-15 11:37:29.734390     Hold        47.500660  Down
1   AZEK 2025-04-16 11:37:29.734390     Sell        47.443783  Down
2   AZEK 2025-04-17 11:37:29.734390      Buy        47.595924  Down
3   AZEK 2025-04-18 11:37:29.734390      Buy        47.621201  Down
4   AZEK 2025-04-19 11:37:29.734390     Sell        47.612469  Down
✅ Completed AI trading & prediction for AZEK!

🚀 Processing FAF...
✅ Finnhub data loaded for FAF (340 rows)
📂 Loading pre-trained model for FAF
1/1 [==============================] - 0s 30ms/step
  Ticker                       Date Buy/Sell  Predicted Price Trend
0    FAF 2025-04-15 11:37:33.586177     Hold        59.958912  Down
1    FAF 2025-04-16 11:37:33.586177     Sell        59.503651 

✅ Completed AI trading & prediction for MOS!

🚀 Processing DXCM...
✅ Finnhub data loaded for DXCM (340 rows)
📂 Loading pre-trained model for DXCM
1/1 [==============================] - 0s 16ms/step
  Ticker                       Date Buy/Sell  Predicted Price Trend
0   DXCM 2025-04-15 11:37:45.577328     Hold        65.427765    Up
1   DXCM 2025-04-16 11:37:45.577328     Sell        65.116318    Up
2   DXCM 2025-04-17 11:37:45.577328      Buy        65.153320    Up
3   DXCM 2025-04-18 11:37:45.577328     Sell        64.967674    Up
4   DXCM 2025-04-19 11:37:45.577328      Buy        65.300728    Up
✅ Completed AI trading & prediction for DXCM!

🚀 Processing NFLX...
✅ Finnhub data loaded for NFLX (340 rows)
📂 Loading pre-trained model for NFLX
1/1 [==============================] - 0s 29ms/step
  Ticker                       Date Buy/Sell  Predicted Price Trend
0   NFLX 2025-04-15 11:37:49.426254     Hold       920.764221  Down
1   NFLX 2025-04-16 11:37:49.426254      Buy       922.7442

✅ Completed AI trading & prediction for EFX!

🚀 Processing ALC...
✅ Finnhub data loaded for ALC (340 rows)
📂 Loading pre-trained model for ALC
1/1 [==============================] - 0s 33ms/step
  Ticker                       Date Buy/Sell  Predicted Price Trend
0    ALC 2025-04-15 11:38:00.394038     Hold        88.159706  Down
1    ALC 2025-04-16 11:38:00.394038     Sell        87.826210  Down
2    ALC 2025-04-17 11:38:00.394038     Sell        87.669945  Down
3    ALC 2025-04-18 11:38:00.394038     Sell        87.531929  Down
4    ALC 2025-04-19 11:38:00.394038     Sell        87.355209  Down
✅ Completed AI trading & prediction for ALC!

🚀 Processing SYK...
✅ Finnhub data loaded for SYK (340 rows)
📂 Loading pre-trained model for SYK
1/1 [==============================] - 0s 36ms/step
  Ticker                       Date Buy/Sell  Predicted Price Trend
0    SYK 2025-04-15 11:38:04.301823     Hold       354.880585  Down
1    SYK 2025-04-16 11:38:04.301823     Sell       353.659271  Dow

✅ Completed AI trading & prediction for HWM!

🚀 Processing CMG...
✅ Finnhub data loaded for CMG (340 rows)
📂 Loading pre-trained model for CMG
1/1 [==============================] - 0s 34ms/step
  Ticker                       Date Buy/Sell  Predicted Price Trend
0    CMG 2025-04-15 11:38:17.061512     Hold        50.272858    Up
1    CMG 2025-04-16 11:38:17.061512      Buy        50.576557    Up
2    CMG 2025-04-17 11:38:17.061512     Sell        50.537613    Up
3    CMG 2025-04-18 11:38:17.061512      Buy        50.781677    Up
4    CMG 2025-04-19 11:38:17.061512      Buy        50.815742    Up
✅ Completed AI trading & prediction for CMG!

🚀 Processing AGI...
✅ Finnhub data loaded for AGI (340 rows)
📂 Loading pre-trained model for AGI
1/1 [==============================] - 0s 35ms/step
  Ticker                       Date Buy/Sell  Predicted Price Trend
0    AGI 2025-04-15 11:38:21.659714     Hold        27.096834    Up
1    AGI 2025-04-16 11:38:21.659714      Buy        27.297241    U

...

1/1 [==============================] - 0s 48ms/step
  Ticker                       Date Buy/Sell  Predicted Price Trend
0   COIN 2025-04-15 12:21:54.759890     Hold       173.257202    Up
1   COIN 2025-04-16 12:21:54.759890      Buy       175.230988    Up
2   COIN 2025-04-17 12:21:54.759890     Sell       175.004547    Up
3   COIN 2025-04-18 12:21:54.759890      Buy       176.847153    Up
4   COIN 2025-04-19 12:21:54.759890      Buy       177.274261    Up
✅ Completed AI trading & prediction for COIN!

🚀 Processing NNN...
✅ Finnhub data loaded for NNN (340 rows)
🛠 Training new model for NNN...
Epoch 1/20
20/20 [==============================] - 3s 13ms/step - loss: 0.0579
Epoch 2/20
20/20 [==============================] - 0s 13ms/step - loss: 0.0164
Epoch 3/20
20/20 [==============================] - 0s 13ms/step - loss: 0.0142
Epoch 4/20
20/20 [==============================] - 0s 14ms/step - loss: 0.0129
Epoch 5/20
20/20 [==============================] - 0s 12ms/step - loss: 0.0119


C:\Users\930140\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 33ms/step
  Ticker                       Date Buy/Sell  Predicted Price Trend
0    NNN 2025-04-15 12:22:10.022219     Hold        39.681808    Up
1    NNN 2025-04-16 12:22:10.022219     Sell        39.588398    Up
2    NNN 2025-04-17 12:22:10.022219      Buy        39.596786    Up
3    NNN 2025-04-18 12:22:10.022219     Sell        39.568996    Up
4    NNN 2025-04-19 12:22:10.022219     Sell        39.532295    Up
✅ Completed AI trading & prediction for NNN!

🚀 Processing TPR...
✅ Finnhub data loaded for TPR (340 rows)
🛠 Training new model for TPR...
Epoch 1/20
20/20 [==============================] - 4s 13ms/step - loss: 0.0864
Epoch 2/20
20/20 [==============================] - 0s 13ms/step - loss: 0.0167
Epoch 3/20
20/20 [==============================] - 0s 13ms/step - loss: 0.0075
Epoch 4/20
20/20 [==============================] - 0s 13ms/step - loss: 0.0063
Epoch 5/20
20/20 [==============================] - 0s 12ms/step - loss: 0.0065
E

C:\Users\930140\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 47ms/step
  Ticker                       Date Buy/Sell  Predicted Price Trend
0    TPR 2025-04-15 12:22:21.492349     Hold        68.962555    Up
1    TPR 2025-04-16 12:22:21.492349      Buy        69.213242    Up
2    TPR 2025-04-17 12:22:21.492349     Sell        68.846611    Up
3    TPR 2025-04-18 12:22:21.492349      Buy        69.201530    Up
4    TPR 2025-04-19 12:22:21.492349     Sell        68.781273    Up
✅ Completed AI trading & prediction for TPR!

🚀 Processing ASX...
✅ Finnhub data loaded for ASX (340 rows)
🛠 Training new model for ASX...
Epoch 1/20
